In [1]:
import requests
import pandas as pd
import json
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path, size="all"):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
        if size != "all" and i >= size:
            break
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('/home/joaopedrosa/Documentos/ufmg/tcc2/data/cell_phones.json.gz', size = 100000)

In [52]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pandas as pd

class EmotionDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        features = self.dataframe.iloc[index]['reviewText']
        rating = self.dataframe.iloc[index]['overall']
        # target = torch.nn.functional.one_hot(torch.tensor(int(rating) - 1), num_classes=5)

        return features, torch.tensor(rating-1).to(torch.long)

In [56]:
batch_size = 4
dataset_train = EmotionDataset(df[:2000])
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
dataset_val = EmotionDataset(df[2000:2200])
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

In [35]:
from steps import train, evaluate, infer

In [36]:
from transformers import BertTokenizer, BertForSequenceClassification

# Defining model and tokenizer
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=5)

# Optimizer and Loss Function - Sticking to standards here
learning_rate = 5e-7
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
device = "cpu"
epochs=5
save_path="models/first_model"

In [ ]:
train(model=model,
      data_loader=dataloader_train,
      val_data_loader=dataloader_val,
      tokenizer=tokenizer,
      loss=loss_fn,
      optimizer=optimizer,
      device=device,
      num_epochs=epochs,
      save_path=save_path)

 32%|█████████████▎                           | 162/500 [04:24<10:08,  1.80s/it]